# Y.Music

# Contents <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
    * [Conclusões](#data_review_conclusions)
* [Etapa 2. Pré-processamento de dados](#data_preprocessing)
    * [2.1 Estilo do cabeçalho](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicatas](#duplicates)
    * [2.4 Conclusões](#data_preprocessing_conclusions)
* [Etapa 3. Testando as hipóteses](#hypotheses)
    * [3.1 Hipótese 1: comparando o comportamento dos usuários em duas cidades](#activity)
    * [3.2 Hipótese 2: música no começo e no fim da semana](#week)
    * [3.3 Hipótese 3: preferências em Springfield e Shelbyville](#genre)
* [Conclusões](#end)


## Introdução <a id='intro'></a>
Sempre que fazemos uma pesquisa, precisamos formular uma hipótese que depois poderemos testar. Às vezes nós aceitamos essas hipóteses; outras vezes, nós as rejeitamos. Para fazer as escolhas certas, um negócio deve ser capaz de entender se está fazendo as suposições certas ou não.

Neste projeto, iremos comparar as preferências musicais dos habitantes de Springfild e Shelbyville. Estudaremos os dados reais do Y.Music para testar a hipótese abaixo e comparar o comportamento do usuário para essas duas cidades.

### Objetivo: 
Teste três hipóteses:
1. A atividade dos usuários é diferente dependendo do dia da semana e da cidade. 
2. Durante as manhãs de segunda-feira, os moradores de Springfield e Shelbyville escutam diferentes gêneros. Isso também é verdadeiro para noites de sexta-feira. 
3. Os ouvintes de Springfield e Shelbyville têm diferentes preferências. Em Springfield, as pessoas preferem pop, enquanto Shelbyville tem mais fãs de rap.

### Etapas 
Os dados sobre o comportamento do usuário é armazenado no arquivo `/datasets/music_project_en.csv`. Não há informação sobre a qualidade dos dados, então, iremos examiná-los antes de testar a hipótese. 

Primeiro, avaliaremos a qualidade dos dados e ver se seus problemas são significativos. Depois, durante o pré-processamento de dados, tentaremos dar conta dos problemas mais críticos problemas.
 
O projeto consistirá em três etapas:
 1. Visão geral dos dados  2. Pré-processamento de dados  3. Testando as hipóteses  
[Voltar ao Índice](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

Abriremos os dados em Y.Music e exploraremos-os. 

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/datasets/music_project_en.csv')

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Obtendo as informações gerais sobre a tabela com um comando:

In [1]:
print(df.describe())
print(df.info())

NameError: name 'df' is not defined

A tabela contém sete colunas. Elas armazenam o mesmo tipo de dado: objetos.

De acordo com a documentação:
- `'userID'` — identificação do usuário
- `'Track'` — título da música
- `'artist'` — nome do artista
- `'genre'` — o gênero
- `'City'` — cidade do usuário
- `'time'` — tempo exato que a música foi tocada
- `'Day'` — dia da semana 

Nós podemos ver três problemas com o estilo nos nomes das colunas:
1. Alguns nomes estão com letra maiúscula, alguns com letra minúscula.
2. Há espaços em alguns nomes.
3. Tipo da coluna time está como object em vez de time

A quantidade de valores das colunas é diferente. Isso significa que os dados contém valores ausentes.


### Conclusões <a id='data_review_conclusions'></a> 

Cada linha na tabela armazena dados sobre uma música que foi tocada. Algumas colunas descrevem a música por si só: seu título, artista e gênero. O restante contém informações sobre o usuário: a cidade de onde eles vêm, a quantidade de vezes que a música foi tocada. 

Está claro que os dados são suficientes para testar as hipóteses. Entretanto, há valores ausentes.

Para seguir adiante, precisamos pré-processar os dados.

[Voltar ao Índice](#back)

## Etapa 2. Pré-processar dados <a id='data_preprocessing'></a>
Corrigiremos a formatação no cabeçalho da coluna e trabalharemos com os valores ausentes. Depois, verificaremos se há duplicatas nos dados.

### Estilo do cabeçalho <a id='header_style'></a>
Exibiremos o cabeçalho da coluna:

In [5]:
print(df.columns)# a lista dos nomes das colunas na tabela df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Mudaremos os nomes das colunas de acordo com as regras da boa prática de estilo:

In [6]:
df=df.rename(
    columns={
        '  userID': 'user_id', 
        'Track': 'track', 
        'artist': 'artist',
        'genre': 'genre',
        '  City  ': 'city',
        'time': 'time', 
        'Day': 'day'
    }
)

In [7]:
print(df.columns)# checando o resultado: a lista dos nomes das colunas

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


[Voltar ao Índice](#back)

### Valores ausentes <a id='missing_values'></a>
Primeiro, encontraremos a quantidade de valores ausentes na tabela. Para fazer isso, usaremos dois métodos pandas:

In [2]:
print(df.isna().sum())

NameError: name 'df' is not defined

Nem todos os valores ausentes afetam a pesquisa. Por exemplo, os valores ausentes na música e artista não é decisivo. Poderemos simplesmente substituí-los por marcadores claros.

Os valores ausentes em 'genre' podem afetar a comparação de preferências musicais de Springfield e Shelbyville. Por se tratar ainda de um primeiro projeto do curso, ainda nao havia ensinamentos suficientes para descobrir as razões pelas quais os dados estão ausentes e tentar compensá-los. Diante dessa impossibilidade neste projeto iremos:
* Preencher os valores ausentes com marcadores
* Avaliar o quanto os valores ausentes podem afetar os seus cálculos

Primeiramente, iremos substituir os valores ausentes em 'track', 'artist', e 'genre' com a string 'unknown'. 

In [9]:
df['track'] = df['track'].fillna('unknown')
df['artist'] = df['artist'].fillna('unknown')
df['genre'] = df['genre'].fillna('unknown')


In [3]:
print(df.isna().sum())

NameError: name 'df' is not defined

[Voltar ao Índice](#back)

### Duplicatas <a id='duplicates'></a>
Encontraremos o número de duplicatas óbvias na tabela usando um comando.

In [11]:
print(df.duplicated().sum())

3826


Chamaremos o método pandas para se livrar das duplicatas óbvias:

In [12]:
df = df.drop_duplicates()

Contaremos as duplicatas óbvias mais uma vez para certificar que removemos todas elas.

In [13]:
print(df.duplicated().sum())

0


Acreditamos que o nome de um gênero pode ser escrito de maneiras diferentes e como alguns erros afetarão também o resultado. Para isso retiraremos as duplicatas implícitas na coluna genre.

Exibiremos a lista de nomes de gêneros únicos, organizados em ordem alfabética. Para fazer isso:
* Recuperaremos o DataFrame da coluna pretendida 
* Aplicaremos um método de escolha para isso
* Para a coluna selecionada, chamaremos o método que irá retornar todos os valores únicos das colunas

In [14]:
print(df['genre'].unique())

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hip' 'jazz' 'postrock' 'latin'
 'classical' 'metal' 'reggae' 'triphop' 'blues' 'instrumental' 'rusrock'
 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'salsa' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'tropical' 'contemporary' 'new' 'soul' 'holiday'
 'german' 'jpop' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'korean'
 'numetal' 'vocal' 'estrada' 'tango' 'loungeelectronic' 'classicmetal'
 'dubstep' 'club' 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french'
 'disco' 'religious' 'hiphop' 'drum' 'extrememetal' 'türkçe'
 'experimental' 'easy' 'metalcore' 'modern' 'argentinetango' 'old' 'swing'
 'breaks' 'eurofolk' 

Olhando a lista e encontramos duplicatas implícitas do gênero hiphop. Esses podem ser nomes escritos incorretamente, ou nomes alternativos para o mesmo gênero.

Vimos as seguintes duplicatas implícitas:
* hip
* hop
* hip-hop

Para se livrar deles, declararemos a função replace_wrong_genres() com dois parâmetros: 
* wrong_genres= — a lista de duplicatas
* correct_genre= — a string com o valor correto

A função deve corrigir os nomes na coluna 'genre' da tabela df, isto é, substituindo cada valor da lista wrong_genres por valores de correct_genre.

In [15]:
def replace_wrong_genres(wrong_genres,correct_genre):
    df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

Chamaremos replace_wrong_genres() e passaremos a ela argumentos para que possamos eliminar as duplicatas implícitas (hip, hop, e hip-hop) e substituiremos por hiphop.

In [16]:
duplicates = ['hip','hop','hip-hop']
correct_genre = 'hiphop'

replace_wrong_genres(duplicates, correct_genre)

In [17]:
print(df['genre'].unique())

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hiphop' 'jazz' 'postrock'
 'latin' 'classical' 'metal' 'reggae' 'triphop' 'blues' 'instrumental'
 'rusrock' 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'salsa' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'tropical' 'contemporary' 'new' 'soul' 'holiday'
 'german' 'jpop' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'korean'
 'numetal' 'vocal' 'estrada' 'tango' 'loungeelectronic' 'classicmetal'
 'dubstep' 'club' 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french'
 'disco' 'religious' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'swing' 'breaks' 'eurofolk'
 'stone

[Voltar ao Índice](#back)

### Conclusões <a id='data_preprocessing_conclusions'></a>
Nós detectamos três problemas com os dados:

- Estilo de cabeçalho incorreto
- Valores ausentes
- Duplicatas óbvias e implícitas

O cabeçalho foi limpo para fazer o processamento da tabela mais simples.

Todos os valores ausentes foram substituídos por 'unkown'. Mas nós ainda temos que ver os se valores ausentes em 'genre' afetará os nossos cálculos.

A ausência de duplicatas deixará os resultados mais precisos e mais fáceis de entender.

Passaremos agora a analisar as hipóteses.

[Voltar ao Índice](#back)

## Etapa 3. Testando hipóteses <a id='hypotheses'></a>

### Hipótese 1: comparando o comportamento dos usuários em duas cidades <a id='activity'></a>

De acordo com a primeira hipótese, usuários de Springfield and Shelbyville escutam música de forma diferente. Teste essa hipótese usando os dados de três dias da semana: segunda-feira, quarta-feira, e sexta-feira.

* Divida os usuários de cada cidade em grupos.
* Compare quantas músicas cada grupo escutou na segunda-feira, quarta-feira e sexta-feira.


Por uma questão de prática, faça cada um desses cálculos separadamente. 

Avalie a atividade do usuário em cada cidade. Agrupe os dados por cidade e encontre o número de músicas tocadas em cada grupo.



In [18]:
print(df.groupby('city').count())

             user_id  track  artist  genre   time    day
city                                                    
Shelbyville    18512  18512   18512  18512  18512  18512
Springfield    42741  42741   42741  42741  42741  42741


Springfield tem mais músicas tocadas do que Shelbyville. Mas isso não quer dizer que os cidadãos de Springfield escutam música com mais frequência. Essa cidade é somente maior, e tem mais usuários.

Agora agruparemos os dados por dia da semana e encontre a quantidade de músicas tocadas na segunda, quarta e sexta-feira.


In [19]:
print(df.groupby('day')['track'].count())


day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


Quarta-feira é o dia mais calmo em geral. Mas se considerarmos as duas cidades separadamente, devemos chegar a uma conclusão diferente.

Você viu como agrupar por cidade ou dia da semana funciona. Criaremos uma função que agrupar os dados por cidade ou dia da semana.

Para criar função number_tracks() iremos calcular o número de músicas tocadas em um determinado dia da semana e em cada cidade usando os parâmetros:
* dia da semana
* nome da cidade

Na função, usaremos a variável para armazenar as linhas da tabela original, onde:
  * o valor da coluna 'day' é igual ao parâmetro dia
  * o valor da coluna 'city' é igual ao parâmetro cidade

Aplicaremos filtros consecutivos com indexação lógica.

Depois, calcularemos os valores da coluna 'user_id' na tabela resultante. Armazenaremos o resultado na nova variável. Retornaremos essa variável da função.

In [20]:

def number_tracks(day, city):
    track_list= df[(df['day'] == day)]
    track_list= track_list[track_list['city'] == city]
    track_list_count= track_list['user_id'].count()
    
    return track_list_count
    

Para comparar quantas músicas cada grupo escutou na segunda-feira, quarta-feira e sexta-feira iremos verificar os dados de ambas as cidades para os três dias. Chamaremos a `number_tracks()` seis vezes, mudando os valores dos parâmetros.

In [21]:
print(number_tracks(day='Monday', city='Springfield'))


15740


In [22]:
print(number_tracks(day='Monday', city='Shelbyville'))


5614


In [23]:
print(number_tracks(day='Wednesday', city='Springfield'))


11056


In [24]:
print(number_tracks(day='Wednesday', city='Shelbyville'))


7003


15945


In [26]:
print(number_tracks(day='Friday', city='Shelbyville'))


5895


Para melhor comparar os resultados usaremos pd.DataFrame para criar uma tabela, onde
* Os nomes das colunas são: ['city', 'monday', 'wednesday', 'friday']`
* Os dados são o resultado que você recebe de number_tracks()

In [27]:
springfield_monday = number_tracks(day='Monday', city='Springfield')
springfield_wednesday = number_tracks(day='Wednesday', city='Springfield')
springfield_friday = number_tracks(day='Friday', city='Springfield')
shelbyville_monday = number_tracks(day='Monday', city='Shelbyville')
shelbyville_wednesday = number_tracks(day='Wednesday', city='Shelbyville')
shelbyville_friday = number_tracks(day='Friday', city='Shelbyville')
dados = [
    ['Springfield', 
    springfield_monday,
    springfield_wednesday,
    springfield_friday],
    ['Shelbyville',
    shelbyville_monday,
    shelbyville_wednesday,
    shelbyville_friday]
]

pd.DataFrame(data = dados, columns =['city', 'monday', 'wednesday', 'friday'])



,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusões**

Os dados revelam diferenças no comportamento dos usuários:

- Em Springfield, a quantidade de músicas tocadas tem seu auge nas segundas e sextas-feiras, enquanto na quarta-feira há uma diminuição na atividade.
- Em Shelbyville, ao contrário, usuários escutam mais música na quarta-feira. A atividade na segunda e sexta-feira é pequena.

Então a primeira hipótese parece ser correta.

[Voltar ao Índice](#back)

### Hipótese 2: música no começo e no fim da semana <a id='week'></a>

De acordo com a segunda hipótese, na segunda-feira de manhã e sexta-feira à noite, habitantes de Springfield escutam gêneros que diferem de alguns usuários de Shelbyville gostam.

Obteremos uma tabela para ambas as cidades.
* Para Springfield — `spr_general`
* Para Shelbyville — `shel_general`

In [28]:
spr_general = df[df['city']=='Springfield']

print(spr_general)


        user_id                          track                   artist  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg   
4      E2DC1FAE                    Soul People               Space Echo   
6      4CB90AA5                           True             Roman Messer   
7      F03E1C1F               Feeling This Way          Polina Griffith   
8      8FA1D3BE                       L’estate              Julia Dalia   
...         ...                            ...                      ...   
65073  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder   
65074  729CBB09                        My Name                   McLean   
65076  C5E3A0D5                      Jalopiina                  unknown   
65077  321D0506                  Freight Train            Chas McDevitt   
65078  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez   

              genre         city      time        day  
1              rock  Springfield  14:07:09 

In [29]:
shel_general = df[df['city']=='Shelbyville']

print(shel_general)




        user_id                              track              artist  \
0      FFB692EC                  Kamigata To Boots    The Mass Missile   
2        20EC38                  Funiculì funiculà         Mario Lanza   
3      A3DD03C9              Dragons in the Sunset          Fire + Ice   
5      842029A1                             Chains            Obladaet   
9      E772D5C0                          Pessimist             unknown   
...         ...                                ...                 ...   
65063  D94F810B        Theme from the Walking Dead  Proyecto Halloween   
65064  BC8EC5CF       Red Lips: Gta (Rover Rework)               Rover   
65065  29E04611                       Bre Petrunko       Perunika Trio   
65066  1B91C621             (Hello) Cloud Mountain     sleepmakeswaves   
65075  D08D4A55  Maybe One Day (feat. Black Spade)         Blu & Exile   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
2  

Escreveremos a função genre_weekday() com quatro parâmetros:
* Uma tabela para dados (`df`)
* O dia da semana (`day`)
* O primeiro carimbo de hora, no formato 'HH:MM' (`time1`)
* O último carimbo de hora, no formato 'HH,MM' (`time2`)

A função deve retornar informações sobre os 15 gêneros mais populares em um determinado dia, dentro do período entre os dois carimbos de hora.

In [30]:

def genre_weekday(df, day, time1, time2):
    
    genre_df = df.loc[df['day']== day]

    
    genre_df = genre_df.loc[genre_df['time']<time2]

    
    genre_df = genre_df.loc[genre_df['time']>time1]

    
    genre_df_count = genre_df.groupby('genre')['user_id'].count()

   
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    
    return genre_df_sorted[:15]

Compararemos os resultados da função `genre_weekday()` para Springfield e Shelbyville na segunda-feira de manhã (de 7hs à 11hs) e na sexta-feira de tarde (das 17hs às 23hs):

In [31]:
genre_weekday(spr_general, 'Monday', '07:00:00', '11:00:00')
  

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: user_id, dtype: int64

In [32]:
genre_weekday(shel_general,'Monday','07:00:00','11:00:00')
    

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: user_id, dtype: int64

In [33]:
genre_weekday(spr_general,'Friday','17:00:00','23:00:00')


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: user_id, dtype: int64

In [34]:
genre_weekday(shel_general,'Friday','17:00:00','23:00:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: user_id, dtype: int64

**Conclusão**

Tendo comparado os 15 gêneros mais ouvidos na segunda-feira de manhã, nós podemos tirar as seguintes conclusões:

1. Usuários de Springfield e Shelbyville escutam músicas semelhantes. Os cinco gêneros mais ouvidos são os mesmos, apenas rock e música eletrônica trocaram de lugar.

2. Em Springfield, a quantidade de valores ausentes acabaram por serem tantos que o valor 'unknown' veio em 10º. Isso significa que valores ausentes tiveram uma considerável porção dos dados, que pode ser a base para questionamentos a confiabilidade das conclusões.

Para sexta-feira à tarde, a situação é parecida. Gêneros individuais variam um pouco, mas no todo, os 15 gêneros mais ouvidos são parecidos para as duas cidades.

Assim, a segunda hipótese foi parcialmente provada:
* Usuários escutam gêneros musicais parecidos no começo e no fim da semana.
* Não há grande diferença entre Springfield e Shelbyville. Nas duas cidades, pop é o gênero mais popular.

No entanto, o número de valores ausentes faz esse resultado ser questionável. Em Springfield, há tantos que eles afetaram o top 15. Se não nos faltassem esses valores, as coisas poderiam ser diferentes.

[Voltar ao Índice](#back)

### Hipótese 3: preferências em Springfield e Shelbyville <a id='genre'></a>

Hipótese: Shelbyville ama rap. Cidadãos de Springfield curtem mais pop.

Agruparemos a tabela spr_general por gênero e encontraremos o número de músicas tocadas para cada gênero com o método count(). Depois organizaremos o resultado em ordem decrescente e armazenaremos em spr_genres.

In [36]:
spr_general = spr_general.groupby('genre').count()


spr_general = spr_general.sort_values(by='genre', ascending=False)


In [37]:
print(spr_general.sort_values('genre').head(10))

             user_id  track  artist  city  time   day
genre                                                
acid               1      1       1     1     1     1
acoustic           3      3       3     3     3     3
action             4      4       4     4     4     4
adult             16     16      16    16    16    16
africa            12     12      12    12    12    12
afrikaans          4      4       4     4     4     4
alternative     1379   1379    1379  1379  1379  1379
ambient          183    183     183   183   183   183
americana          7      7       7     7     7     7
animated           2      2       2     2     2     2


Agora faremos o mesmo com os dados de Shelbyville.

In [38]:

shel_general = shel_general.groupby('genre').count()


shel_general = shel_general.sort_values(by='genre', ascending=False)


In [39]:
print(shel_general.sort_values('genre').head(10))

                user_id  track  artist  city  time  day
genre                                                  
acoustic              2      2       2     2     2    2
adult                 8      8       8     8     8    8
africa                4      4       4     4     4    4
alternative         649    649     649   649   649  649
ambient              64     64      64    64    64   64
americana             1      1       1     1     1    1
anime                29     29      29    29    29   29
arabesk               2      2       2     2     2    2
arabic                1      1       1     1     1    1
argentinetango        7      7       7     7     7    7


**Conclusão**

A hipótese foi parcialmente provada:
* Música pop é o gênero mais em Springfield, como esperado.
* Entretanto, música pop acabou por ser igualmente popular em Springfield e Shelbyville, e rap não estava no top 5 em nenhuma cidade.


[Voltar ao Índice](#back)

# Conclusões <a id='end'></a>

Nós testamos as três hipóteses seguintes:

1. A atividade do usuário varia dependendo do dia da semana e da cidade. 
2. Nas segundas-feiras de manhã, os habitantes de Springfield e Shelbyville escutam diferentes gêneros. Isso também é verdadeiro para noites de sexta-feira. 
3. Ouvintes de Springfield e Shelbyville têm preferências diferentes. Tanto em Springfield como Shellbyville, eles preferem pop.

Depois de analisar os dados, nós concluímos:

1. A atividade do usuário em Springfield e Shelbyville depende do dia da semana, embora as cidades variam de formas diferentes. 

A primeira hipótese é totalmente aceita.

2. As preferências musicais não variam significativamente ao decorrer da semana tanto em Springfield como em Shelbyville. Nós podemos ver pequenas diferenças na ordem nas segundas-feiras, mas:
* Em Springfield e Shelbyville, as pessoas escutam mais música pop.

Então nós podemos aceitar essa hipótese. Nós devemos também ter em mente que o resultado pode ter sido diferente se não fosse pelos valores ausentes.

3. Acontece que preferências musicais dos usuários de Springfield e Shelbyville são bastante parecidas.

A terceira hipótese foi rejeitada. Se há alguma diferença nas preferências, ela não pode ser vista nesses dados.

[Voltar ao Índice](#back)